In [140]:
import qutip as qt
from qutip_qip.operations.gates import hadamard_transform as hadamard

import numpy as np
from matplotlib import pyplot as plt

import jax
import jax.numpy as jnp
from diffrax import Dopri5, Dopri8, Tsit5, PIDController

In [141]:
from optimize import optimize_pulses
from time_interval import TimeInterval
from objective import Objective

# Toy model

In [142]:
initial = qt.qeye(2)
target  = hadamard()

initial = qt.sprepost(initial, initial.dag())
target  = qt.sprepost(target , target.dag() )

In [143]:
σx = qt.sigmax()
σy = qt.sigmay()
σz = qt.sigmaz()

ω, Δ, γ, π = 0.1, 1.0, 0.1, np.pi

Hd = 1/2 * (ω * σz + Δ * σx)

H_d =  qt.liouvillian(H=Hd, c_ops=[np.sqrt(γ) * qt.sigmam()])

In [144]:
interval = TimeInterval(tlist=[0, 2*π])

In [145]:
def sin(t, α):
    return α[0] * np.sin(α[1] * t + α[2])

def grad_sin(t, α, idx):
    if idx==0: return np.sin(α[1] * t + α[2])
    if idx==1: return α[0] * np.cos(α[1] * t + α[2]) * t
    if idx==2: return α[0] * np.cos(α[1] * t + α[2])
    if idx==3: return α[0] * np.cos(α[1] * t + α[2]) * α[1] # w.r.t. time

In [146]:
sin_x = lambda t, p: sin(t, p)
sin_y = lambda t, q: sin(t, q)
sin_z = lambda t, r: sin(t, r)

In [147]:
Hc  = [σx, σy, σz]
H_c = [qt.liouvillian(H) for H in Hc]

H = [ H_d,
     [H_c[0], sin_x, {"grad": grad_sin}],
     [H_c[1], sin_y, {"grad": grad_sin}],
     [H_c[2], sin_z, {"grad": grad_sin}]]

In [148]:
p_init = [1, 1, 0]
q_init = [1, 1, 0]
r_init = [1, 1, 0]

# Default integrator settings - local search only - fix time

GOAT

In [149]:
res_goat = optimize_pulses(
    objectives = [Objective(initial, H, target)],
    pulse_options={
        "p": {
            "guess":  p_init, # p0 * sin(p1 * t + p2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init, # q0 * sin(q1 * t + q2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init, # r0 * sin(r1 * t + r2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    algorithm_kwargs = {
        "alg": "GOAT",
        "fid_err_targ": 0.01,
        "method": "basinhopping",
        "disp": True,
        "max_iter": 0, # global optimizer steps
        "seed": 1,
    },
    
)

minimizer step, infidelity: 0.27608
minimizer step, infidelity: 0.26606
minimizer step, infidelity: 0.24657
minimizer step, infidelity: 0.19541
minimizer step, infidelity: 0.11592
minimizer step, infidelity: 0.09042
minimizer step, infidelity: 0.07068
minimizer step, infidelity: 0.06814
minimizer step, infidelity: 0.06529
minimizer step, infidelity: 0.06451
minimizer step, infidelity: 0.06267
minimizer step, infidelity: 0.05990
minimizer step, infidelity: 0.05549
minimizer step, infidelity: 0.05311
minimizer step, infidelity: 0.05289
minimizer step, infidelity: 0.05285
minimizer step, infidelity: 0.05276
minimizer step, infidelity: 0.05264
minimizer step, infidelity: 0.05248
minimizer step, infidelity: 0.05234
minimizer step, infidelity: 0.05186
minimizer step, infidelity: 0.05125
minimizer step, infidelity: 0.05072
minimizer step, infidelity: 0.05029
minimizer step, infidelity: 0.05026
minimizer step, infidelity: 0.05022
minimizer step, infidelity: 0.05017
minimizer step, infidelity: 

In [150]:
res_goat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:27:26
- Number of objectives: 1
- Final fidelity error: 0.04403656960277009
- Final parameters: [[1.0, 0.6462771162438175, 2.1425609700529074], [0.9996894160117288, 0.990898249375864, 0.004499661412899851], [0.8444724950457054, 2.379880939421669e-09, 0.2993300539999815]]
- Number of iterations: 1
- Reason for termination: ['requested number of basinhopping iterations completed successfully']
- Ended at 2023-11-10 17:27:33 (6.7618s)

JAX

In [151]:
def sin_jax(t, α):
    return α[0] * jnp.sin(α[1] * t + α[2])

In [152]:
@jax.jit
def sin_x_jax(t, p, **kwargs):
    return sin_jax(t, p)

@jax.jit
def sin_y_jax(t, q, **kwargs):
    return sin_jax(t, q)

@jax.jit
def sin_z_jax(t, r, **kwargs):
    return sin_jax(t, r)

In [153]:
H_jax = [ H_d,
         [H_c[0], sin_x_jax],
         [H_c[1], sin_y_jax],
         [H_c[2], sin_z_jax]]

In [154]:
res_joat = optimize_pulses(
    objectives = [Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess":  p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    algorithm_kwargs = {
        "alg": "JOAT",
        "fid_err_targ": 0.01,
        "method": "basinhopping",
        "disp": True,
        "max_iter": 0,
        "seed": 1,
    },
)

minimizer step, infidelity: 0.27609
minimizer step, infidelity: 0.26607
minimizer step, infidelity: 0.24655
minimizer step, infidelity: 0.19537
minimizer step, infidelity: 0.11597
minimizer step, infidelity: 0.09035
minimizer step, infidelity: 0.07066
minimizer step, infidelity: 0.06813
minimizer step, infidelity: 0.06528
minimizer step, infidelity: 0.06450
minimizer step, infidelity: 0.06266
minimizer step, infidelity: 0.05988
minimizer step, infidelity: 0.05548
minimizer step, infidelity: 0.05310
minimizer step, infidelity: 0.05288
minimizer step, infidelity: 0.05284
minimizer step, infidelity: 0.05275
minimizer step, infidelity: 0.05263
minimizer step, infidelity: 0.05247
minimizer step, infidelity: 0.05233
minimizer step, infidelity: 0.05185
minimizer step, infidelity: 0.05124
minimizer step, infidelity: 0.05073
minimizer step, infidelity: 0.05028
minimizer step, infidelity: 0.05026
minimizer step, infidelity: 0.05021
minimizer step, infidelity: 0.05017
minimizer step, infidelity: 

In [155]:
res_joat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:27:33
- Number of objectives: 1
- Final fidelity error: 0.04455113743096623
- Final parameters: [[0.999553068793097, 0.9677108939313139, 1.3896276385936615], [0.7368374722530612, 0.7802926694405331, 1.5571836032831659], [0.3879646737896883, 0.0, 1.7601127919800026]]
- Number of iterations: 1
- Reason for termination: ['requested number of basinhopping iterations completed successfully']
- Ended at 2023-11-10 17:28:04 (30.8379s)

# Default integrator settings - local search only - variable time

GOAT

In [156]:
res_joat = optimize_pulses(
    objectives = [Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess":  p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    time_options = {
        "guess": interval.evo_time,
        "bounds": (0, 2*interval.evo_time),
    },
    algorithm_kwargs = {
        "alg": "JOAT",
        "fid_err_targ": 0.01,
        "method": "basinhopping",
        "disp": True,
        "max_iter": 0,
        "seed": 1,
    },
)

minimizer step, infidelity: 0.27596
minimizer step, infidelity: 0.26577
minimizer step, infidelity: 0.24572
minimizer step, infidelity: 0.19266
minimizer step, infidelity: 0.11669
minimizer step, infidelity: 0.10373
minimizer step, infidelity: 0.08367
minimizer step, infidelity: 0.07925
minimizer step, infidelity: 0.06900
minimizer step, infidelity: 0.06096
minimizer step, infidelity: 0.05621
minimizer step, infidelity: 0.05442
minimizer step, infidelity: 0.05415
minimizer step, infidelity: 0.05395
minimizer step, infidelity: 0.05351
minimizer step, infidelity: 0.05318
minimizer step, infidelity: 0.05249
minimizer step, infidelity: 0.05209
minimizer step, infidelity: 0.05184
minimizer step, infidelity: 0.05163
minimizer step, infidelity: 0.05153
minimizer step, infidelity: 0.05145
minimizer step, infidelity: 0.05140
minimizer step, infidelity: 0.05137
minimizer step, infidelity: 0.05134
minimizer step, infidelity: 0.05133
minimizer step, infidelity: 0.05132
minimizer step, infidelity: 

In [157]:
res_goat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:27:26
- Number of objectives: 1
- Final fidelity error: 0.04403656960277009
- Final parameters: [[1.0, 0.6462771162438175, 2.1425609700529074], [0.9996894160117288, 0.990898249375864, 0.004499661412899851], [0.8444724950457054, 2.379880939421669e-09, 0.2993300539999815]]
- Number of iterations: 1
- Reason for termination: ['requested number of basinhopping iterations completed successfully']
- Ended at 2023-11-10 17:27:33 (6.7618s)

JAX

In [158]:
res_joat = optimize_pulses(
    objectives = [Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess":  p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    time_options = {
        "guess": interval.evo_time,
        "bounds": (0, 2*interval.evo_time),
    },
    algorithm_kwargs = {
        "alg": "JOAT",
        "fid_err_targ": 0.01,
        "method": "basinhopping",
        "disp": True,
        "max_iter": 0,
        "seed": 1,
    },
)

minimizer step, infidelity: 0.27596
minimizer step, infidelity: 0.26577
minimizer step, infidelity: 0.24572
minimizer step, infidelity: 0.19266
minimizer step, infidelity: 0.11669
minimizer step, infidelity: 0.10373
minimizer step, infidelity: 0.08367
minimizer step, infidelity: 0.07925
minimizer step, infidelity: 0.06900
minimizer step, infidelity: 0.06096
minimizer step, infidelity: 0.05621
minimizer step, infidelity: 0.05442
minimizer step, infidelity: 0.05415
minimizer step, infidelity: 0.05395
minimizer step, infidelity: 0.05351
minimizer step, infidelity: 0.05318
minimizer step, infidelity: 0.05249
minimizer step, infidelity: 0.05209
minimizer step, infidelity: 0.05184
minimizer step, infidelity: 0.05163
minimizer step, infidelity: 0.05153
minimizer step, infidelity: 0.05145
minimizer step, infidelity: 0.05140
minimizer step, infidelity: 0.05137
minimizer step, infidelity: 0.05134
minimizer step, infidelity: 0.05133
minimizer step, infidelity: 0.05132
minimizer step, infidelity: 

In [159]:
res_joat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:28:28
- Number of objectives: 1
- Final fidelity error: 0.02492020332425142
- Final parameters: [[1.0, 0.9485234732252087, 2.0175236634621174], [0.9999985248371083, 0.9174976046839698, 0.9463379704948467], [0.9605607074573544, 1.0, 0.9800389764496323], [3.846593105352696]]
- Number of iterations: 1
- Reason for termination: ['requested number of basinhopping iterations completed successfully']
- Ended at 2023-11-10 17:28:51 (23.11s)

# Same integrator settings - global search

GOAT

In [164]:
res_goat = optimize_pulses(
    objectives = [Objective(initial, H, target)],
    pulse_options={
        "p": {
            "guess":  p_init, # p0 * sin(p1 * t + p2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init, # q0 * sin(q1 * t + q2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init, # r0 * sin(r1 * t + r2)
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    time_options = {
        "guess": interval.evo_time,
        "bounds": (0, 2*interval.evo_time),
    },
    algorithm_kwargs = {
        "alg": "GOAT",
        "method": "dual_annealing",
        "fid_err_targ": 0.01,
        "disp": True,
        "max_iter": 100,
        "seed": 1,
    },
    integrator_kwargs = {
        "atol": 1e-6,
        "rtol": 1e-6,
        "method": "dop853",
    },
)

optimizer step, infidelity: 0.14197, took 0.08 seconds
minimizer step, infidelity: 0.04040
minimizer step, infidelity: 0.03003
minimizer step, infidelity: 0.02714
minimizer step, infidelity: 0.02697
optimizer step, infidelity: 0.02697, took 0.95 seconds
optimizer step, infidelity: 0.01949, took 11.79 seconds
optimizer step, infidelity: 0.01761, took 0.00 seconds
minimizer step, infidelity: 0.01382
minimizer step, infidelity: 0.01345
minimizer step, infidelity: 0.01334
minimizer step, infidelity: 0.01329
optimizer step, infidelity: 0.01329, took 0.53 seconds


In [165]:
res_goat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:30:34
- Number of objectives: 1
- Final fidelity error: 0.01328501973909172
- Final parameters: [[0.16275264534575196, 0.14666474720376024, 4.548448377528713], [0.3352717573102745, 0.726879603957995, 4.0035211375470405], [0.9583942759452505, 0.7371399968016987, 5.8964259935535885], [2.4718503482401557]]
- Number of iterations: 100
- Reason for termination: ['Maximum number of iteration reached']
- Ended at 2023-11-10 17:31:05 (30.7651s)

JAX

In [166]:
res_joat = optimize_pulses(
    objectives = [Objective(initial, H_jax, target)],
    pulse_options={
        "p": {
            "guess":  p_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "q": {
            "guess":  q_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        },
        "r": {
            "guess":  r_init,
            "bounds": [(-1, 1), (0, 1), (0, 2*np.pi)],
        }
    },
    time_interval = interval,
    time_options = {
        "guess": interval.evo_time,
        "bounds": (0, 2*interval.evo_time),
    },
    algorithm_kwargs = {
        "alg": "JOAT",
        "method": "dual_annealing",
        "fid_err_targ": 0.01,
        "disp": True,
        "max_iter": 100,
        "seed": 1,
    },
    integrator_kwargs = {
        "stepsize_controller": PIDController(
            atol = 1e-6,
            rtol = 1e-6,
        ),
        "solver": Dopri8(),
    },
)

optimizer step, infidelity: 0.13791, took 1.75 seconds
minimizer step, infidelity: 0.11816
minimizer step, infidelity: 0.08924
minimizer step, infidelity: 0.08275
minimizer step, infidelity: 0.08171
minimizer step, infidelity: 0.08100
minimizer step, infidelity: 0.08044
minimizer step, infidelity: 0.07963
minimizer step, infidelity: 0.07847
minimizer step, infidelity: 0.07733
minimizer step, infidelity: 0.07577
minimizer step, infidelity: 0.07294
minimizer step, infidelity: 0.06826
minimizer step, infidelity: 0.06606
minimizer step, infidelity: 0.06275
minimizer step, infidelity: 0.05744
minimizer step, infidelity: 0.04918
minimizer step, infidelity: 0.04540
minimizer step, infidelity: 0.03948
minimizer step, infidelity: 0.03308
minimizer step, infidelity: 0.03166
minimizer step, infidelity: 0.02954
minimizer step, infidelity: 0.02701
minimizer step, infidelity: 0.02656
minimizer step, infidelity: 0.02583
minimizer step, infidelity: 0.02488
minimizer step, infidelity: 0.02422
minimizer

In [167]:
res_joat

Control Optimization Result
--------------------------
- Started at 2023-11-10 17:31:05
- Number of objectives: 1
- Final fidelity error: 0.004609634541730921
- Final parameters: [[0.4037346765474621, 0.6412245011923677, 1.1238995677340033], [0.014759329784488393, 0.17776074280489226, 1.998389686022864], [0.8417012947771373, 0.33787128389855775, 1.4778811366267417], [1.227293080135112]]
- Number of iterations: 71
- Reason for termination: fid_err_targ reached
- Ended at 2023-11-10 17:31:47 (42.0568s)